# Group Comparison analysis pRF propertis 

In [12]:
# Load the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import shutil
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from CFandPRF import load_prf, filter_prf, PRFModel, source_eccentricity

In [13]:
# Define the main variables 
target = 'V3'
source = 'V1'
hemis = ['lh', 'rh']
atlas = 'manual'
delineation = 'manualdelin'
denoising = 'nordic'
visual_areas = ['V2', 'V3', 'V4', 'LO']
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM'
pkl_path = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
freesurfer = f"{pkl_path}/freesurfer"
poag = [f'sub-{i:02}' for i in range(2, 21)]
hc = [f'sub-{i:02}' for i in range(21, 47)]
subjects =  hc+ poag 
cumulative = os.path.join(MAIN_PATH,'CDFs')
sigmas = []
tasks = {"RET": {"label": "RET","dir": "RET","color": {"HC": "blue", "POAG": "red"}}, 
         "RET2": {"label": "RET2","dir": "RET2","color": {"HC": "green", "POAG": "salmon"}}, 
         "REST": {"label": "REST","dir": "RestingState","color": {"HC": "black", "POAG": "gray"}}}
groups = {"HC": hc, "POAG": poag}

In [14]:
oct_values_combined = {
    'sub-02': 73, 'sub-04': 57, 'sub-05': 59, 'sub-06': 72, 'sub-07': 86.5,
    'sub-08': 71, 'sub-09': 65, 'sub-10': 64.5, 'sub-11': 59.5, 'sub-12': 56,
    'sub-13': 63, 'sub-14': 73.5, 'sub-15': 74.5, 'sub-16': 65.5, 'sub-17': 70.5,
    'sub-18': 71, 'sub-19': 55, 'sub-20': 64, 'sub-21': 72.5, 'sub-22': 89,
    'sub-23': 68, 'sub-25': 105, 'sub-26': 87, 'sub-27': 100.5, 'sub-28': 69,
    'sub-29': 82.5, 'sub-30': 100, 'sub-31': 89.5, 'sub-32': 80.5, 'sub-33': 97.5,
    'sub-34': 100.5, 'sub-35': 100, 'sub-36': 103, 'sub-37': 95.5, 'sub-38': 100,
    'sub-39': 104.5, 'sub-40': 101, 'sub-41': 101, 'sub-42': 102, 'sub-43': 110.5,
    'sub-44': 105, 'sub-45': 100.5, 'sub-46': 94.5}
hfa_values_combined = {
    'sub-02': -15.83, 'sub-04': -20.76, 'sub-05': -9.25, 'sub-06': 0.62, 'sub-07': -2.88,
    'sub-08': -6.1, 'sub-09': -4.35, 'sub-10': -11.54, 'sub-11': -2.08, 'sub-12': -8.68,
    'sub-13': -9.76, 'sub-14': -8.06, 'sub-15': -1.9, 'sub-16': -1.68, 'sub-17': -5.46,
    'sub-18': -12.41, 'sub-19': -7.78, 'sub-20': -15.24, 'sub-21': -1.94, 'sub-22': -0.91,
    'sub-23': -7.54, 'sub-25': -0.38, 'sub-26': -0.38, 'sub-27': 1.42, 'sub-28': 0.29,
    'sub-29': -0.14, 'sub-30': 0.19, 'sub-31': 0.37, 'sub-32': 1.64, 'sub-33': -0.12,
    'sub-34': -2.16, 'sub-35': 0.07, 'sub-36': 0.84, 'sub-37': -2.01, 'sub-38': -0.39,
    'sub-39': 0.5, 'sub-40': 0.9, 'sub-41': -1.01, 'sub-42': 1.17, 'sub-43': -1.08,
    'sub-44': 1.23, 'sub-45': -0.28, 'sub-46': 0.7}
hfa_values_besteye = {
    'sub-02': -1.12, 'sub-04': -18.43, 'sub-05': -7.5, 'sub-06': 0.95, 'sub-07': -2.1,
    'sub-08': -0.09, 'sub-09': -2.56, 'sub-10': -8.37, 'sub-11': -2.05, 'sub-12': -4.7,
    'sub-13': -4.24, 'sub-14': -3.52, 'sub-15': 0.52, 'sub-16': -1.1, 'sub-17': -4.56,
    'sub-18': -0.92, 'sub-19': -6.11, 'sub-20': -2.36, 'sub-21': 0.38, 'sub-22': 0.24,
    'sub-23': -2.67, 'sub-25': -0.11, 'sub-26': 0.71, 'sub-27': 1.75, 'sub-28': 0.6,
    'sub-29': 0.2, 'sub-30': 0.97, 'sub-31': 0.51, 'sub-32': 1.73, 'sub-33': 0.26,
    'sub-34': -1.58, 'sub-35': 0.45, 'sub-36': 1.25, 'sub-37': -0.87, 'sub-38': 0.84,
    'sub-39': 1.35, 'sub-40': 1.69, 'sub-41': -0.89, 'sub-42': 1.76, 'sub-43': -0.95,
    'sub-44': 1.91, 'sub-45': -0.22, 'sub-46': 0.97}
oct_values_besteye = {
    'sub-02': 101, 'sub-04': 58, 'sub-05': 64, 'sub-06': 73, 'sub-07': 91,
    'sub-08': 78, 'sub-09': 76, 'sub-10': 66, 'sub-11': 60, 'sub-12': 63,
    'sub-13': 66, 'sub-14': 75, 'sub-15': 86, 'sub-16': 71, 'sub-17': 72,
    'sub-18': 86, 'sub-19': 60, 'sub-20': 71, 'sub-21': 77, 'sub-22': 95,
    'sub-23': 75, 'sub-25': 108, 'sub-26': 89, 'sub-27': 101, 'sub-28': 93,
    'sub-29': 86, 'sub-30': 101, 'sub-31': 91, 'sub-32': 82, 'sub-33': 99,
    'sub-34': 101, 'sub-35': 104, 'sub-36': 112, 'sub-37': 105, 'sub-38': 101,
    'sub-39': 105, 'sub-40': 102, 'sub-41': 101, 'sub-42': 104, 'sub-43': 111,
    'sub-44': 108, 'sub-45': 103, 'sub-46': 96}
oct_values_worsteye = {
    'sub-02': 79, 'sub-04': 56, 'sub-05': 54, 'sub-06': 71, 'sub-07': 82,
    'sub-08': 64, 'sub-09': 54, 'sub-10': 63, 'sub-11': 59, 'sub-12': 49,
    'sub-13': 60, 'sub-14': 72, 'sub-15': 63, 'sub-16': 60, 'sub-17': 69,
    'sub-18': 56, 'sub-19': 50, 'sub-20': 57, 'sub-21': 68, 'sub-22': 83,
    'sub-23': 61, 'sub-25': 102, 'sub-26': 85, 'sub-27': 100, 'sub-28': 45,
    'sub-29': 79, 'sub-30': 99, 'sub-31': 88, 'sub-32': 79, 'sub-33': 96,
    'sub-34': 100, 'sub-35': 96, 'sub-36': 94, 'sub-37': 86, 'sub-38': 99,
    'sub-39': 104, 'sub-40': 100, 'sub-41': 101, 'sub-42': 100, 'sub-43': 110,
    'sub-44': 102, 'sub-45': 98, 'sub-46': 93}
hfa_values_worsteye = {
    'sub-02': -30.54, 'sub-04': -23.08, 'sub-05': -11, 'sub-06': 0.3, 'sub-07': -3.67,
    'sub-08': -12.11, 'sub-09': -6.14, 'sub-10': -14.72, 'sub-11': -2.12, 'sub-12': -12.66,
    'sub-13': -15.29, 'sub-14': -12.6, 'sub-15': -4.32, 'sub-16': -2.25, 'sub-17': -6.36,
    'sub-18': -23.9, 'sub-19': -9.45, 'sub-20': -28.11, 'sub-21': -4.25, 'sub-22': -2.06,
    'sub-23': -12.4, 'sub-25': -0.65, 'sub-26': -1.47, 'sub-27': 1.08, 'sub-28': -0.02,
    'sub-29': -0.48, 'sub-30': -0.59, 'sub-31': 0.23, 'sub-32': 1.55, 'sub-33': -0.49,
    'sub-34': -2.73, 'sub-35': -0.31, 'sub-36': 0.44, 'sub-37': -3.15, 'sub-38': -1.62,
    'sub-39': -0.36, 'sub-40': 0.12, 'sub-41': -1.12, 'sub-42': 0.58, 'sub-43': -1.21,
    'sub-44': 0.55, 'sub-45': -0.34, 'sub-46': 0.43}

In [ ]:
def sigma(subj, visual_area, task_name):
    sigma_vals = []
    task_dir = tasks[task_name]["dir"]
    for hemi in hemis:
        path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_dir}/{visual_area}/best_fits_prf.csv'
        if not os.path.exists(path):
            continue
        df = pd.read_csv(path)
        df["Best Sigma Finer"] = pd.to_numeric(df["Best Sigma Finer"], errors='coerce')
        df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')
        df = df.dropna(subset=["Best Sigma Finer", "Best Variance Explained Finer"])
        df = df[(df["Best Sigma Finer"] > 0.05) & (df["Best Variance Explained Finer"] > 0.3)]
        sigma_vals.extend(df["Best Sigma Finer"].values)
    return sigma_vals

def individual_cdfs(group, color, label, visual_area, task_name):
    cdfs = []
    included_subjects = 0
    for subj in group:
        sigma_vals = sigma(subj, visual_area, task_name)
        included_subjects += 1
        sorted_sigma = np.sort(sigma_vals)
        cdf = np.linspace(0, 1, len(sorted_sigma))
        plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.4)
        grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
        cdfs.append(grid_cdf)
    if cdfs:
        median = np.median(cdfs, axis=0)
        plt.plot(x, median, color=color, linewidth=2.5, label=f"{label} Median (N={included_subjects})")
    return cdfs

# Setup output directories
def setup_output_dirs(cumulative, denoising):
    sigma_output_dir = os.path.join(cumulative, f"groupcomparison_{denoising}_jpg")
    ecc_output_dir = os.path.join(cumulative, f"eccentricity_comparison_{denoising}_jpg")
    os.makedirs(sigma_output_dir, exist_ok=True)
    os.makedirs(ecc_output_dir, exist_ok=True)
    return sigma_output_dir, ecc_output_dir

# Plot individual sigma CDFs
def plot_sigma_cdfs(visual_area, x, cmap, norm, sigma_output_dir):
    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            #if (group_name, task_name) not in included_combinations:
            #    continue

            plt.figure(figsize=(6, 6))
            cdfs, included_subjects = [], 0

            for subj in group_subjects:
                sigma_vals = sigma(subj, visual_area, task_name)
                if not sigma_vals:
                    continue
                included_subjects += 1
                sorted_sigma = np.sort(sigma_vals)
                cdf = np.linspace(0, 1, len(sorted_sigma))
                hfa_value = hfa_values_worsteye.get(subj, 0)
                color = cmap(norm(hfa_value))
                plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.7)
                grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} Median (N={included_subjects})"
                plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)

            plt.title(f"{visual_area} - {group_name} - {task_name}")
            plt.xlabel("Connective Field Sigma (mm)")
            plt.ylabel("Cumulative Probability")
            plt.grid(True)
            plt.legend()
            plot_path = os.path.join(sigma_output_dir, f"{visual_area}_{group_name}_{task_name}.jpg")
            plt.savefig(plot_path, dpi=300)
            plt.close()

# Plot combined sigma CDFs
def plot_combined_sigma_cdfs(visual_area, x, sigma_output_dir):
    plt.figure(figsize=(7, 7))
    plotted_anything = False

    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            #if (group_name, task_name) not in included_combinations:
            #    continue

            cdfs, included_subjects = [], 0

            for subj in group_subjects:
                sigma_vals = sigma(subj, visual_area, task_name)
                if not sigma_vals:
                    continue
                included_subjects += 1
                sorted_sigma = np.sort(sigma_vals)
                cdf = np.linspace(0, 1, len(sorted_sigma))
                grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} (N={included_subjects})"
                plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)
                plotted_anything = True

    plt.title(f"{visual_area} - All Groups & Tasks Comparison {denoising}")
    plt.xlabel("Connective Field Sigma (mm)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    if plotted_anything:
        plt.legend()
    else:
        plt.text(0.5, 0.5, "No data available", ha='center', va='center', transform=plt.gca().transAxes)

    summary_plot_path = os.path.join(sigma_output_dir, f"{visual_area}_all_comparison.jpg")
    plt.savefig(summary_plot_path, dpi=300)
    plt.close()

# Plot eccentricity CDFs
def plot_eccentricity_cdfs(visual_area, x_ecc, ecc_output_dir):
    plt.figure(figsize=(7, 7))
    plotted_anything = False

    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            #if (group_name, task_name) not in included_combinations:
            #    continue

            cdfs, included_subjects = [], 0

            for subj in group_subjects:
                ecc_vals = []
                for hemi in hemis:
                    path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_info["dir"]}/{visual_area}/best_fits_prf.csv'
                    if not os.path.exists(path):
                        continue
                    df = pd.read_csv(path)
                    df = df.dropna(subset=["Source Eccentricity", "Best Variance Explained Finer"])
                    df = df[df["Best Variance Explained Finer"] > 0.3]
                    ecc_vals.extend(df["Source Eccentricity"].values)

                if ecc_vals:
                    included_subjects += 1
                    sorted_ecc = np.sort(ecc_vals)
                    cdf = np.linspace(0, 1, len(sorted_ecc))
                    grid_cdf = np.interp(x_ecc, sorted_ecc, cdf, left=0, right=1)
                    cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} (N={included_subjects})"
                plt.plot(x_ecc, median, color=task_info["color"][group_name], linewidth=2.5, label=label)
                plotted_anything = True

    plt.title(f"{visual_area} - All Groups & Tasks Comparison (Eccentricity)")
    plt.xlabel("Eccentricity (deg)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    if plotted_anything:
        plt.legend()
    else:
        plt.text(0.5, 0.5, "No data available", ha='center', va='center', transform=plt.gca().transAxes)

    summary_plot_path = os.path.join(ecc_output_dir, f"{visual_area}_all_comparison_ecc.jpg")
    plt.savefig(summary_plot_path, dpi=300)
    plt.close()

# Plot the eccentricity CDFs vs PRFs
def eccentricity_cf_prf(visual_area, ecc_output_dir):
    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            # print(task_name)
            for subj in group_subjects:
                cf_ecc_vals = []
                prf_ecc_vals = []

                for hemi in hemis:
                    # Load CF eccentricity from CSV
                    path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_info["dir"]}/{visual_area}/best_fits_prf.csv'
                    if not os.path.exists(path):
                        continue

                    df = pd.read_csv(path)
                    df = df.dropna(subset=["Source Eccentricity", "Best Variance Explained Finer"])
                    df = df[df["Best Variance Explained Finer"] > 0.3]

                    # Load pRF eccentricity
                    try:
                        prf_ecc_dict = source_eccentricity(
                                subj, hemi, pkl_path, atlas, denoising, task_name, freesurfer)
                    except FileNotFoundError:
                        print(f"File not found for {subj}")

                    # Match vertices
                    for idx, row in df.iterrows():
                        vertex = row["Target Vertex Index"]
                        cf_ecc = row["Source Eccentricity"]
                        prf_ecc_dict = source_eccentricity(
                                subj, hemi, pkl_path, atlas, denoising, task_name, freesurfer)
                        prf_ecc = prf_ecc_dict.get(vertex, None)
                        if prf_ecc is not None:
                            cf_ecc_vals.append(cf_ecc)
                            prf_ecc_vals.append(prf_ecc)

                if cf_ecc_vals and prf_ecc_vals:
                    plt.figure(figsize=(7, 7))
                    plt.scatter(cf_ecc_vals, prf_ecc_vals, alpha=0.5)
                    plt.xlabel("CF Eccentricity (deg)")
                    plt.ylabel("pRF Eccentricity (deg)")
                    plt.title(f"{visual_area} - {subj} - {group_name} {task_name}")
                    plt.grid(True)
                    plt.plot([0, max(cf_ecc_vals + prf_ecc_vals)], [0, max(cf_ecc_vals + prf_ecc_vals)], 'r--', label='y = x')
                    plt.legend()

                    plot_path = os.path.join(ecc_output_dir, f"{visual_area}_{subj}_{group_name}_{task_name}_cf_vs_prf_ecc.jpg")
                    plt.savefig(plot_path, dpi=300)
                    plt.close()


sigma_output_dir, ecc_output_dir = setup_output_dirs(cumulative, denoising)

cmap = cm.Reds
norm = mcolors.Normalize(vmin=-30, vmax=5)
x = np.linspace(0, 10, 500)
x_ecc = np.linspace(0, 10, 500)

for visual_area in visual_areas:
    plot_sigma_cdfs(visual_area, x, cmap, norm, sigma_output_dir)
    plot_combined_sigma_cdfs(visual_area, x, sigma_output_dir)
    eccentricity_cf_prf(visual_area, ecc_output_dir)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
def sigma(subj, visual_area, task_name):
    sigma_vals = []
    task_dir = tasks[task_name]["dir"]
    for hemi in hemis:
        path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_dir}/{visual_area}/best_fits_prf.csv'
        if not os.path.exists(path):
            continue
        df = pd.read_csv(path)
        df["Best Sigma Finer"] = pd.to_numeric(df["Best Sigma Finer"], errors='coerce')
        df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')
        df = df.dropna(subset=["Best Sigma Finer", "Best Variance Explained Finer"])
        df = df[(df["Best Sigma Finer"] > 0.05) & (df["Best Variance Explained Finer"] > 0.3)]
        sigma_vals.extend(df["Best Sigma Finer"].values)
    return sigma_vals

def individual_cdfs(group, color, label, visual_area, task_name):
    cdfs = []
    included_subjects = 0
    for subj in group:
        sigma_vals = sigma(subj, visual_area, task_name)
        included_subjects += 1
        sorted_sigma = np.sort(sigma_vals)
        cdf = np.linspace(0, 1, len(sorted_sigma))
        plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.4)
        grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
        cdfs.append(grid_cdf)
    if cdfs:
        median = np.median(cdfs, axis=0)
        plt.plot(x, median, color=color, linewidth=2.5, label=f"{label} Median (N={included_subjects})")
    return cdfs

In [ ]:
output_dir = os.path.join(cumulative, f"groupcomparison_{denoising}_jpg")
os.makedirs(output_dir, exist_ok=True)
# Define colormap and normalization for HFA values
cmap = cm.Reds
norm = mcolors.Normalize(vmin=-30, vmax=5)  # Adjust based on your HFA value range

# Only include specific (group, task) combinations
included_combinations = {("POAG", "RET"),("HC", "RET"),("HC", "RET2")}

for visual_area in visual_areas:
    x = np.linspace(0, 10, 500)

    # A. Individual group/task combinations
    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            if (group_name, task_name) not in included_combinations:
                continue
            plt.figure(figsize=(6, 6))
            cdfs = []
            included_subjects = 0

            for subj in group_subjects:
                sigma_vals = sigma(subj, visual_area, task_name)
                if not sigma_vals:
                    continue
                included_subjects += 1
                sorted_sigma = np.sort(sigma_vals)
                cdf = np.linspace(0, 1, len(sorted_sigma))
                # Get HFA value for coloring 
                hfa_value = hfa_values_worsteye.get(subj, 0)
                color = cmap(norm(hfa_value)) # Map HFA value to color
                plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.7)               
                grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} Median (N={included_subjects})"
                plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)

            plt.title(f"{visual_area} - {group_name} - {task_name}")
            plt.xlabel("Connective Field Sigma (mm)")
            plt.ylabel("Cumulative Probability")
            plt.grid(True)
            plt.legend()
            plot_path = os.path.join(output_dir, f"{visual_area}_{group_name}_{task_name}.jpg")
            plt.savefig(plot_path, dpi=300)
            plt.close()
    
    # B. All groups/tasks in one plot
    plt.figure(figsize=(7, 7))
    plotted_anything = False

    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            if (group_name, task_name) not in included_combinations:
                continue
            cdfs = []
            included_subjects = 0

            for subj in group_subjects:
                sigma_vals = sigma(subj, visual_area, task_name)
                if not sigma_vals:
                    continue
                included_subjects += 1
                sorted_sigma = np.sort(sigma_vals)
                cdf = np.linspace(0, 1, len(sorted_sigma))
                grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} (N={included_subjects})"
                plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)
                plotted_anything = True

    plt.title(f"{visual_area} - All Groups & Tasks Comparison {denoising}")
    plt.xlabel("Connective Field Sigma (mm)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    if plotted_anything:
        plt.legend()
    else:
        plt.text(0.5, 0.5, "No data available", ha='center', va='center', transform=plt.gca().transAxes)

    summary_plot_path = os.path.join(output_dir, f"{visual_area}_all_comparison.jpg")
    plt.savefig(summary_plot_path, dpi=300)
    plt.close()

In [ ]:
ecc_output_dir = os.path.join(cumulative, f"eccentricity_comparison_{denoising}_jpg")
os.makedirs(ecc_output_dir, exist_ok=True)

x_ecc = np.linspace(0, 10, 500)

for visual_area in visual_areas:
    plt.figure(figsize=(7, 7))
    plotted_anything = False

    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            if (group_name, task_name) not in included_combinations:
                continue
            cdfs = []
            included_subjects = 0

            for subj in group_subjects:
                ecc_vals = []
                task_dir = task_info["dir"]
                for hemi in hemis:
                    path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_dir}/{visual_area}/best_fits_prf.csv'
                    if not os.path.exists(path):
                        continue
                    df = pd.read_csv(path)
                    df["Source Eccentricity"] = pd.to_numeric(df["Source Eccentricity"], errors='coerce')
                    df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')
                    df = df.dropna(subset=["Source Eccentricity", "Best Variance Explained Finer"])
                    df = df[df["Best Variance Explained Finer"] > 0.3]
                    ecc_vals.extend(df["Source Eccentricity"].values)

                if ecc_vals:
                    included_subjects += 1
                    sorted_ecc = np.sort(ecc_vals)
                    cdf = np.linspace(0, 1, len(sorted_ecc))
                    grid_cdf = np.interp(x_ecc, sorted_ecc, cdf, left=0, right=1)
                    cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} (N={included_subjects})"
                plt.plot(x_ecc, median, color=task_info["color"][group_name], linewidth=2.5, label=label)
                plotted_anything = True

    plt.title(f"{visual_area} - All Groups & Tasks Comparison (Eccentricity)")
    plt.xlabel("Eccentricity (deg)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    plt.legend()
    if plotted_anything:
        plt.legend()
    else:
        plt.text(0.5, 0.5, "No data available", ha='center', va='center', transform=plt.gca().transAxes)

    summary_plot_path = os.path.join(ecc_output_dir, f"{visual_area}_all_comparison_ecc.jpg")
    plt.savefig(summary_plot_path, dpi=300)
    plt.close()